In [2]:
# Importing Libraries
import imageio.v3 as iio # pip install imageio[ffmpeg]
import matplotlib.pyplot as plt
import cv2
import numpy as np
import open3d as o3d # pip install open3d

## 2. Frame Extraction

In [3]:
def load_video_frames(video_path, frame_interval=10, display_frames=True):
    # Load frames from a video file at specified intervals.

    frame_count = 0
    frames = []  # List to store frames for visualization

    try:
        print(f"Opening video file: {video_path}")
        
        # Iterate through frames in the video file
        for frame in iio.imiter(video_path):
            if frame_count % frame_interval == 0:
                frames.append(frame)  # Store frame in list
                
                # Display frame if display_frames is True
                if display_frames:
                    plt.imshow(frame)
                    plt.title(f'Frame {frame_count}')
                    plt.axis('off')
                    plt.show()
                
            frame_count += 1

    except FileNotFoundError:
        print(f"Error: Video file not found at {video_path}")
        return [], 0
    except Exception as e:
        print(f"An error occurred while processing the video: {e}")
        return [], 0

    print(f"\nFinished processing video.")
    print(f"Total frames iterated: {frame_count}.")
    
    return frames

frames = load_video_frames('../videos/bedroomvideo.mp4', frame_interval=10, display_frames=False)

Opening video file: ../videos/bedroomvideo.mp4

Finished processing video.
Total frames iterated: 237.


## 5. Essential/ Fundamental Matrix Computation

Both matrices describe the epipolar geometry between two images of the same scene taken from different viewpoints. Epipolar geometry defines the constraint that corresponding points must satisfy. If you have a point x in the first image, its corresponding point x' in the second image must lie on a specific line called the epiline. Both matrices capture this relationship, but they differ based on camera calibration.

The Fundamental Matrix acts as a bridge between the 2D pixel coordinates of corresponding points in two different images, capturing the geometric constraints imposed by the 3D scene and the camera positions, even when you don't know the camera's exact internal details. It's a fundamental tool for tasks like finding correct feature matches, estimating camera motion, and ultimately reconstructing the 3D scene.

- Fundamental Matrix (F):
  - **What it is**: A 3x3 matrix that relates corresponding points between two images in pixel coordinates.
  - **Information encoded**: It contains information about the camera's relative rotation and translation (extrinsic parameters) and the intrinsic parameters (like focal length, principal point) of both cameras.
  - **Equation**: It satisfies the epipolar constraint: x'^T * F * x = 0, where x and x' are the homogeneous coordinates of the matching points in pixels.
  - **When to use**: Use the Fundamental Matrix when the cameras are uncalibrated, meaning you don't know their intrinsic parameters.
  - **Computation**: Typically requires at least 8 pairs of corresponding points (using the 8-point algorithm) or 7 pairs (7-point algorithm). OpenCV's function often uses robust methods like RANSAC or LMedS which handle outliers well using many more points.

If you are recording video with a phone and have not performed a specific camera calibration procedure to find its intrinsic matrix (K), then using the Fundamental Matrix (F) is the recommended approach. You treat the camera as uncalibrated.

- Essential Matrix (E):
  - **What it is**: A 3x3 matrix that relates corresponding points between two images in normalized image coordinates (independent of camera intrinsics).
  - **Information encoded**: It contains only information about the camera's relative rotation (R) and translation (t), up to a scale factor. It does not include camera intrinsic information.
  - **Equation**: It satisfies the epipolar constraint in normalized coordinates: x_norm'^T * E * x_norm = 0.
  - **When to use**: Use the Essential Matrix when the cameras are calibrated, meaning you know their intrinsic parameters (focal length, principal point - often represented in a camera matrix K).
  - **Computation**: Requires at least 5 pairs of corresponding points (using the 5-point algorithm), though robust methods in OpenCV use more points.
  - **Relation to F**: E = K'^T * F * K, where K and K' are the camera intrinsic matrices for the two views. If the camera is the same for both views, K' = K.

In [4]:
def compute_fundamental_matrix(pts1, pts2, method=cv2.FM_RANSAC, ransacReprojThreshold=1.0, confidence=0.999, maxIters=2000):
    """
    Computes the Fundamental Matrix (F) and filters inliers between two sets of points.

    Args:
        pts1 (np.ndarray): NumPy array of N points from image 1, shape (N, 1, 2), dtype=float32/64.
        pts2 (np.ndarray): NumPy array of N points from image 2, shape (N, 1, 2), dtype=float32/64.
        method (int, optional): Method for computation (e.g., cv2.FM_RANSAC, cv2.FM_LMEDS).
                                Defaults to cv2.FM_RANSAC.
        ransacReprojThreshold (float, optional): RANSAC reprojection threshold (pixels). Defaults to 1.0.
        confidence (float, optional): RANSAC confidence level. Defaults to 0.999.
        maxIters (int, optional): Maximum RANSAC iterations. Defaults to 2000.

    Returns:
        tuple: A tuple containing:
            - F (np.ndarray or None): The computed 3x3 Fundamental Matrix, or None if failed.
            - pts1_inliers (np.ndarray or None): Inlier points from image 1, or None if failed.
            - pts2_inliers (np.ndarray or None): Inlier points from image 2, or None if failed.
            - status_mask (np.ndarray or None): The mask indicating inliers (1) and outliers (0).
    """
    if pts1 is None or pts2 is None or len(pts1) < 8 or len(pts2) < 8:
        print("Error: Not enough points to compute Fundamental Matrix.")
        return None, None, None, None

    try:
        # Compute the Fundamental Matrix using the chosen method
        F, mask = cv2.findFundamentalMat(pts1, pts2,
                                         method=method,
                                         ransacReprojThreshold=ransacReprojThreshold,
                                         confidence=confidence,
                                         maxIters=maxIters)

        if F is None or mask is None:
            print("Warning: findFundamentalMat returned None.")
            return None, None, None, None

        # Filter points using the mask to get inliers
        pts1_inliers = pts1[mask.ravel() == 1]
        pts2_inliers = pts2[mask.ravel() == 1]
        num_inliers = len(pts1_inliers)
        num_total = len(pts1)

        print(f"Fundamental Matrix computed. Inliers: {num_inliers} / {num_total}")

        return F, pts1_inliers, pts2_inliers, mask

    except cv2.error as e:
        print(f"OpenCV Error during findFundamentalMat: {e}")
        return None, None, None, None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None, None, None

In [5]:
# Driver code
# 1. Assume you have your matched points (replace with your actual data)
num_points = 150
pts1_matched = np.random.rand(num_points, 1, 2).astype(np.float32) * 500
translation_demo = np.array([30, 10], dtype=np.float32).reshape(1, 1, 2)
noise_demo = np.random.randn(num_points, 1, 2).astype(np.float32) * 3
# Add some outliers
outlier_indices = np.random.choice(num_points, size=int(num_points * 0.2), replace=False)
pts2_matched = pts1_matched + translation_demo + noise_demo
pts2_matched[outlier_indices] = np.random.rand(len(outlier_indices), 1, 2).astype(np.float32) * 500

# 2. Call the function
fundamental_matrix, pts1_inliers, pts2_inliers, inlier_mask = compute_fundamental_matrix(
    pts1_matched,
    pts2_matched,
    ransacReprojThreshold=1.5, # Example threshold adjustment
    confidence=0.99
)

# 3. Check the result and use the outputs
if fundamental_matrix is not None:
    print("Successfully computed F and filtered inliers.")
    print(f"Shape of pts1_inliers: {pts1_inliers.shape}")
    print(f"Shape of pts2_inliers: {pts2_inliers.shape}")

    # Now you can pass 'fundamental_matrix', 'pts1_inliers', 'pts2_inliers'
    # to the next stage (e.g., camera pose estimation)

else:
    print("Failed to compute Fundamental Matrix or find sufficient inliers.")

Fundamental Matrix computed. Inliers: 65 / 150
Successfully computed F and filtered inliers.
Shape of pts1_inliers: (65, 1, 2)
Shape of pts2_inliers: (65, 1, 2)


## 6. Camera Pose Estimation

The goal of this stage is to determine the relative motion between the two camera views where you matched features. This motion is described by:

- **Rotation (R)**: A 3x3 matrix describing how the camera orientation changed between the two shots.
- **Translation (t)**: A 3x1 vector describing how the camera position changed between the two shots. Note that the translation vector t can only be determined up to a scale factor. This means you know the direction of motion but not the absolute distance moved.

- Input Matrix: cv2.recoverPose technically requires the Essential Matrix (E) as its main input, not the Fundamental Matrix (F). It also needs the corresponding inlier points from the previous stage (pts1_inliers, pts2_inliers) and the camera intrinsic matrix (K).
- Handling the Uncalibrated Case (Starting with F): Since you computed F (because the phone camera was treated as uncalibrated), you need a way to get E to use recoverPose. The relationship is E = K^T * F * K. But K is unknown!
  - The Workaround: You need to assume a plausible K matrix. A common approach is:
Set the principal point (cx, cy) to the image center (e.g., width/2, height/2).
  - Estimate or guess the focal length (fx, fy). Sometimes fx = fy = image_width is used as a starting guess, or a typical value for phone cameras (e.g., 500-1000 pixels) might be assumed.
  - Compute E: Calculate E = K_assumed.T @ F @ K_assumed.
  - Use recoverPose: Call cv2.recoverPose using this computed E, the same assumed K, and your inlier points.
  - Important: You must document this assumption about K in your report. The resulting pose (especially translation t) will be relative to the scale defined by your assumed K and F.
- Chirality Problem: Mathematically, decomposing the E matrix yields four possible solutions for the rotation (R) and translation (t). However, only one of these solutions is physically correct – the one where the reconstructed 3D points lie in front of both cameras.
  - How recoverPose Solves It: The cv2.recoverPose function handles this automatically! It takes your inlier points (pts1_inliers, pts2_inliers) and the K matrix, triangulates the points for each of the four possible (R, t) combinations, and counts how many points end up in front of both camera views. It then returns the R and t corresponding to the hypothesis with the most positive depth points, effectively resolving the chirality ambiguity. Your report should explain this concept.

In [6]:
def estimate_camera_pose(pts1, pts2, F, K_assumed):
    """
    Estimates the relative camera pose (Rotation and Translation) from the
    Fundamental Matrix (F) and corresponding inlier points using cv2.recoverPose.

    Args:
        pts1 (np.ndarray): Inlier points from image 1, shape (N, 1, 2) or (N, 2).
        pts2 (np.ndarray): Inlier points from image 2, shape (N, 1, 2) or (N, 2).
        F (np.ndarray): The 3x3 Fundamental Matrix computed previously.
        K_assumed (np.ndarray): The assumed 3x3 camera intrinsic matrix.
                           Crucial for converting F to E and for chirality check.

    Returns:
        tuple: A tuple containing:
            - R (np.ndarray or None): The 3x3 estimated Rotation matrix, or None if failed.
            - t (np.ndarray or None): The 3x1 estimated Translation vector (up to scale), or None if failed.
            - points_valid_mask (np.ndarray or None): Mask indicating points used by recoverPose for the final check.
    """
    if F is None or K_assumed is None or pts1 is None or pts2 is None or len(pts1) < 5:
         print("Error: Insufficient input for pose estimation.")
         return None, None, None

    # Ensure points are in the correct shape (N, 2) for recoverPose input consistency check
    # Although recoverPose can sometimes handle (N, 1, 2), ensuring (N, 2) is safer.
    pts1_rp = np.reshape(pts1, (-1, 2)).astype(np.float64)
    pts2_rp = np.reshape(pts2, (-1, 2)).astype(np.float64)

    try:
        # 1. Compute Essential Matrix from F and assumed K
        E = K_assumed.T @ F @ K_assumed
        print("Computed Essential Matrix (E) from F and assumed K.")
        # print("E:\n", E) # Uncomment to view

        # 2. Recover Pose (R, t) using cv2.recoverPose
        # This function internally handles the chirality check using the points and K
        points, R, t, mask_pose = cv2.recoverPose(E, pts1_rp, pts2_rp, K_assumed)

        num_valid_points = cv2.countNonZero(mask_pose) if mask_pose is not None else 0
        print(f"Pose recovered. Number of points consistent with the pose: {num_valid_points} / {len(pts1_rp)}")

        if num_valid_points < 5: # Need at least a few points for a reliable pose
             print("Warning: Very few points consistent with the recovered pose. Result might be unreliable.")
             # Depending on requirements, you might return None here
             # return None, None, None

        return R, t, mask_pose # R is 3x3, t is 3x1

    except cv2.error as e:
        print(f"OpenCV Error during recoverPose: {e}")
        return None, None, None
    except Exception as e:
        print(f"An unexpected error occurred during pose estimation: {e}")
        return None, None, None

In [7]:
# Driver code

# 1. Assume you have results from the previous stage:
# fundamental_matrix, pts1_inliers, pts2_inliers (from compute_fundamental_matrix)

# --- Placeholder results (replace with your actual data) ---
# F_computed = np.random.rand(3, 3) # Replace with actual F
# pts1_in = np.random.rand(50, 1, 2).astype(np.float32) * 500 # Replace with actual inliers
# pts2_in = np.random.rand(50, 1, 2).astype(np.float32) * 500 # Replace with actual inliers
# If the compute_fundamental_matrix function was just run:
F_computed = fundamental_matrix
pts1_in = pts1_inliers
pts2_in = pts2_inliers
# -- Placeholder end ---

# 2. Define your assumed K matrix (Example!)
#    You MUST justify your choice/estimation of K in your report.
image_width = 640 # Example image width
image_height = 480 # Example image height
assumed_cx = image_width / 2
assumed_cy = image_height / 2
assumed_focal = image_width # A common starting guess if unknown
K_assumed = np.array([
    [assumed_focal, 0, assumed_cx],
    [0, assumed_focal, assumed_cy],
    [0, 0, 1]
], dtype=np.float64)
print("\nUsing Assumed K matrix:\n", K_assumed)

# 3. Call the pose estimation function
if F_computed is not None and pts1_in is not None and pts2_in is not None:
    R_estimated, t_estimated, pose_mask = estimate_camera_pose(pts1_in, pts2_in, F_computed, K_assumed)

    # 4. Check results
    if R_estimated is not None and t_estimated is not None:
        print("\nSuccessfully estimated camera pose.")
        print("Rotation (R):\n", R_estimated)
        print("Translation (t) (up to scale):\n", t_estimated)
        # You now have the relative rotation R and translation t between the two camera views.
    else:
        print("\nFailed to estimate camera pose.")
else:
    print("\nSkipping pose estimation due to missing F or inlier points from previous stage.")


Using Assumed K matrix:
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Computed Essential Matrix (E) from F and assumed K.
Pose recovered. Number of points consistent with the pose: 10 / 65

Successfully estimated camera pose.
Rotation (R):
 [[ 0.99907164  0.00923915  0.04207716]
 [-0.00984555  0.99985031  0.01422728]
 [-0.04193941 -0.01462834  0.99901306]]
Translation (t) (up to scale):
 [[ 0.15971083]
 [-0.2872955 ]
 [-0.94443303]]


## 7. 3D Point Triangulation and Scene Visualisation

- Concept: Triangulation is the process of determining the 3D coordinates of a point in space. You can do this if you have observed the projection of that point in at least two images taken from different, known camera viewpoints. Given the 2D coordinates of the point in each image (pts1, pts2) and the camera poses (relative rotation R and translation t between the views, plus the camera intrinsics K), you can find the intersection of the "rays" that go from each camera center through its corresponding 2D image point. This intersection point is the estimated 3D location of the point.
- Projection Matrices (P1, P2): To use cv2.triangulatePoints, you need the 3x4 projection matrix for each camera view. This matrix maps 3D world points (in homogeneous coordinates) to 2D image points (in homogeneous coordinates).
  - Camera 1 (Reference): We typically assume the first camera is at the origin of the world coordinate system. Its projection matrix P1 is formed using the intrinsic matrix K and a standard pose [I | 0] (Identity rotation, Zero translation): P1 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
  - Camera 2 (Relative Pose): The second camera's projection matrix P2 is formed using the same K (assuming the same camera or the assumed K from Stage 6) and the relative rotation R and translation t calculated in Stage 6: P2 = K @ np.hstack((R, t))
- cv2.triangulatePoints: This OpenCV function takes the two projection matrices (P1, P2) and the corresponding 2D points from both images (pts1, pts2) as input.
  - Input Format: Note that cv2.triangulatePoints expects the 2D points as 2xN arrays (2 rows, N columns). You'll likely need to transpose your (N, 1, 2) or (N, 2) point arrays.
  - Output Format: It returns the 3D points in homogeneous coordinates as a 4xN array (4 rows, N columns). To get the standard 3D Cartesian coordinates, you need to divide the first three rows by the fourth row.

In [8]:
def triangulate_points(pts1, pts2, K, R, t):
    """
    Triangulates 3D points from corresponding 2D points in two views.

    Args:
        pts1 (np.ndarray): Inlier points from image 1, shape (N, 2) or (N, 1, 2), dtype=float64.
        pts2 (np.ndarray): Inlier points from image 2, shape (N, 2) or (N, 1, 2), dtype=float64.
        K (np.ndarray): The 3x3 camera intrinsic matrix (assumed or calibrated).
        R (np.ndarray): The 3x3 estimated Rotation matrix (from view 1 to view 2).
        t (np.ndarray): The 3x1 estimated Translation vector (from view 1 to view 2).

    Returns:
        np.ndarray or None: Array of triangulated 3D points in Cartesian coordinates (N, 3),
                           or None if input is invalid or triangulation fails.
    """
    if pts1 is None or pts2 is None or K is None or R is None or t is None:
        print("Error: Missing inputs for triangulation.")
        return None
    if len(pts1) != len(pts2):
        print("Error: Number of points in pts1 and pts2 must match.")
        return None
    if len(pts1) == 0:
        print("Error: No points provided for triangulation.")
        return None

    # Reshape points to (N, 2) if needed and ensure correct type (float64 preferred by some examples)
    pts1_tri = np.reshape(pts1, (-1, 2)).astype(np.float64)
    pts2_tri = np.reshape(pts2, (-1, 2)).astype(np.float64)

    # Construct projection matrices P = K @ [R|t]
    # P1: Camera 1 at origin
    P1 = K @ np.hstack((np.eye(3, dtype=np.float64), np.zeros((3, 1), dtype=np.float64)))
    # P2: Camera 2 pose relative to Camera 1
    P2 = K @ np.hstack((R.astype(np.float64), t.astype(np.float64)))

    try:
        # Triangulate points - Input points need to be 2xN
        # Output is 4xN homogeneous coordinates
        points_4d_hom = cv2.triangulatePoints(P1, P2, pts1_tri.T, pts2_tri.T)

        # Convert from homogeneous to Cartesian coordinates (divide by 4th element)
        # Avoid division by zero or very small numbers
        w = points_4d_hom[3, :]
        # Set points with invalid W to NaN or handle appropriately
        # points_3d = points_4d_hom[:3, :] / w # Basic conversion

        # Safer conversion: handle potential zero W
        points_3d = np.zeros((3, points_4d_hom.shape[1]))
        valid_w = np.abs(w) > 1e-6 # Threshold to avoid division by near-zero
        points_3d[:, valid_w] = points_4d_hom[:3, valid_w] / w[valid_w]
        # Decide how to handle invalid points (e.g., filter them out later)
        # For now, we keep the structure, invalid points might be at origin or NaN

        # Transpose to get shape (N, 3)
        points_3d = points_3d.T
        num_valid = np.sum(valid_w)
        print(f"Successfully triangulated points. {num_valid} points with valid depth.")
        return points_3d

    except cv2.error as e:
        print(f"OpenCV Error during triangulatePoints: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during triangulation: {e}")
        return None

Explanation: Scene Visualisation

- Goal: To display the calculated 3D points to visually inspect the reconstructed scene structure.
- Tools: Your project allows libraries like Matplotlib, Open3D, or CloudCompare.
- Interactivity: The visualization must be interactive, allowing you to rotate, pan, and zoom the 3D view.
  - Matplotlib (mplot3d): Can create basic 3D scatter plots. Interactivity (rotation/zoom) often works well in specific environments like Jupyter Notebook using the %matplotlib notebook backend.
  - Open3D: A library specifically designed for 3D data processing and visualization. It generally provides more robust and feature-rich interactive visualization windows suitable for point clouds. This is likely the better choice to meet the "interactive" requirement robustly.

In [9]:
def visualize_3d_scene_open3d(points_3d, point_colors=None, window_name="Open3D Scene Visualization"):
    """
    Visualizes the 3D point cloud interactively using Open3D.
    Opens a separate, blocking window.
    Args:
        points_3d (list or np.ndarray): A list of [x, y, z] points.
        point_colors (list or np.ndarray, optional): Optional RGB color values (0.0-1.0).
        window_name (str, optional): Title for the visualization window.
    """
    if not len(points_3d):
        print("No points to visualize.")
        return
    
    # Create point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.asarray(points_3d))
    
    if point_colors is not None:
        pcd.colors = o3d.utility.Vector3dVector(np.asarray(point_colors))
    else:
        pcd.paint_uniform_color([0.5, 0.5, 0.5])  # Default to gray
    
    # Create coordinate frame to visualize the origin
    coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
    
    try:
        # Try using the visualization approach directly from the documentation
        from open3d.visualization import draw_geometries
        draw_geometries(
            [pcd, coordinate_frame],
            window_name=window_name,
            width=800, 
            height=600
        )
    except Exception as e:
        print(f"Visualization error: {e}")
        print("Saving point cloud to file instead.")
        temp_file = "point_cloud_3d.ply"
        o3d.io.write_point_cloud(temp_file, pcd)
        print(f"Point cloud saved to {temp_file}. Please open it with an external viewer.")
    
    print("Visualization complete.")

In [ ]:
# Driver code

# 1. Assume you have results from previous stages:
# R_estimated, t_estimated (from estimate_camera_pose)
# pts1_inliers, pts2_inliers (inliers from F or E computation)
# K_matrix (assumed or calibrated K used in pose estimation)

# --- Placeholder results (replace with your actual data) ---
# Use the variables from the previous example call if available
# R_est = R_estimated
# t_est = t_estimated
# pts1_for_tri = pts1_inliers
# pts2_for_tri = pts2_inliers
# K_matrix = K_assumed
# Example values if previous variables not available:
R_est_demo = np.array([[0.99, 0.01, 0.0], [-0.01, 0.99, 0.0], [0.0, 0.0, 1.0]])
t_est_demo = np.array([[0.1], [0.0], [0.01]])
K_matrix_demo = np.array([[800.0, 0, 320.0], [0, 800.0, 240.0], [0, 0, 1.0]])
# Generate some dummy points consistent with a simple scene
N_pts_demo = 50
true_3d = np.random.rand(N_pts_demo, 3) * np.array([2, 2, 5]) + np.array([-1, -1, 3]) # Points in front
P1_demo = K_matrix_demo @ np.hstack((np.eye(3), np.zeros((3, 1))))
P2_demo = K_matrix_demo @ np.hstack((R_est_demo, t_est_demo))
pts1_demo = (P1_demo @ np.hstack((true_3d, np.ones((N_pts_demo, 1)))).T).T
pts2_demo = (P2_demo @ np.hstack((true_3d, np.ones((N_pts_demo, 1)))).T).T
pts1_demo = (pts1_demo[:, :2] / pts1_demo[:, 2:]) + np.random.randn(N_pts_demo, 2) * 0.5 # Add noise
pts2_demo = (pts2_demo[:, :2] / pts2_demo[:, 2:]) + np.random.randn(N_pts_demo, 2) * 0.5 # Add noise

R_est = R_est_demo
t_est = t_est_demo
pts1_for_tri = pts1_demo
pts2_for_tri = pts2_demo
K_matrix = K_matrix_demo
# --- End Placeholder ---

# 2. Triangulate the points
if R_est is not None and t_est is not None and K_matrix is not None and pts1_for_tri is not None and pts2_for_tri is not None:
    print("\n--- Starting Triangulation ---")
    points_3d_scene = triangulate_points(pts1_for_tri, pts2_for_tri, K_matrix, R_est, t_est)

    if points_3d_scene is not None and len(points_3d_scene) > 0:
        print("\n--- Starting Visualization ---")
        visualize_3d_scene_open3d(points_3d_scene)
    else:
        print("Triangulation failed or no points to visualize.")
else:
     print("\nSkipping triangulation and visualization due to missing inputs from previous stages.")


--- Starting Triangulation ---
Successfully triangulated points. 50 points with valid depth.

--- Starting Visualization ---
Visualization complete.
